In [8]:
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import numpy as np 
from torch.utils.data import DataLoader,Dataset,random_split
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [1]:
import torch as t
import numpy as np
t.cuda.set_device(8)

import warnings
warnings.filterwarnings("ignore")
device = t.device('cuda' if t.cuda.is_available() else 'cpu')

In [2]:
audio_dir = "/raid/amana/lavish_multi_model/emotion_detection/data/raw_audio"

In [3]:
from transformers import AutoProcessor, MCTCTModel
from datasets import load_dataset
import torch

In [4]:
processor = AutoProcessor.from_pretrained("speechbrain/m-ctc-t-large")
model = MCTCTModel.from_pretrained("speechbrain/m-ctc-t-large").to(device)

Some weights of the model checkpoint at speechbrain/m-ctc-t-large were not used when initializing MCTCTModel: ['ctc_head.bias', 'ctc_head.weight']
- This IS expected if you are initializing MCTCTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MCTCTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 processor = AutoProcessor.from_pretrained("speechbrain/m-ctc-t-large")                       │
│ ❱ 2 model = MCTCTModel.from_pretrained("speechbrain/m-ctc-t-large").to(device)                   │
│   3                                                                                              │
│                                                                                                  │
│ /raid/amana/miniconda3/envs/lamol/lib/python3.8/site-packages/transformers/modeling_utils.py:188 │
│ 6 in to                                                                                          │
│                                                                                                  │
│   1883 │   │   │   │   " model has already been set to the correct devices and casted to the co  │
│   1884 │   │   │   )                                                                             │
│   1885 │   │   else:                                                                             │
│ ❱ 1886 │   │   │   return super().to(*args, **kwargs)                                            │
│   1887 │                                                                                         │
│   1888 │   def half(self, *args):                                                                │
│   1889 │   │   # Checks if the model has been loaded in 8-bit                                    │
│                                                                                                  │
│ /raid/amana/miniconda3/envs/lamol/lib/python3.8/site-packages/torch/nn/modules/module.py:673 in  │
│ to                                                                                               │
│                                                                                                  │
│    670 │   │   │   │   │   │   │   non_blocking, memory_format=convert_to_format)                │
│    671 │   │   │   return t.to(device, dtype if t.is_floating_point() or t.is_complex() else No  │
│    672 │   │                                                                                     │
│ ❱  673 │   │   return self._apply(convert)                                                       │
│    674 │                                                                                         │
│    675 │   def register_backward_hook(                                                           │
│    676 │   │   self, hook: Callable[['Module', _grad_t, _grad_t], Union[None, Tensor]]           │
│                                                                                                  │
│ /raid/amana/miniconda3/envs/lamol/lib/python3.8/site-packages/torch/nn/modules/module.py:387 in  │
│ _apply                                                                                           │
│                                                                                                  │
│    384 │                                                                                         │
│    385 │   def _apply(self, fn):                                                                 │
│    386 │   │   for module in self.children():                                                    │
│ ❱  387 │   │   │   module._apply(fn)                                                             │
│    388 │   │                                                                                     │
│    389 │   │   def compute_should_use_set_data(tensor, tensor_applied):                          │
│    390 │   │   │   if torch._has_compatible_shallow_copy_type(tensor, tensor_applied):           │
│                                                            

In [ ]:
emotion_label = {'anger' : 1, 'disgust' : 2, 'fear' : 3,
                 'joy' : 4, 'neutral' : 5, 'sadness' : 6, 'surprise' : 0}

In [ ]:
final_dataset = []
for a in dataset:
    inputs = processor(a[0], sampling_rate=16000, return_tensors="pt").to(device)
    with t.no_grad():
        outputs = model(**inputs).extract_features
        final_dataset.append([outputs.squeeze(0),emotion_label[a[1]]])

In [ ]:
inputs = processor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
predicted_ids = torch.argmax(logits, dim=-1)